<a href="https://colab.research.google.com/github/patronasxdxd/CTF-hardhat/blob/master/deeplearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import os

def scrape_google_images(query):
    url = f"https://www.google.com/search?q={query}&tbm=isch&tbs=isz:l"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    images = []
    for img in soup.find_all("img"):
        img_url = img.get("src")
        if img_url and img_url.startswith("http"):
            images.append(img_url)

    return images


# Example usage
kanye_images = scrape_google_images("kanye west face front")
ariana_images = scrape_google_images("ariana grande Face front")
kanye_lookalike_images = scrape_google_images("micheal b jordan face front")
ariana_lookalike_images = scrape_google_images("bella hadid face front")







In [3]:
import numpy as np
import cv2
import requests
from tempfile import NamedTemporaryFile
from io import BytesIO
from PIL import Image
from google.colab.patches import cv2_imshow
import os

def get_image_from_url(url):
    response = requests.get(url)
    image = Image.open(BytesIO(response.content))
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    return image

def process_and_save_images(image_urls, dir_name, max_faces=10):
    faces_found = 0
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)

    for url in image_urls:
        if faces_found >= max_faces:
            break
        try:
            img = get_image_from_url(url)
            img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            faces = faceCascade.detectMultiScale(img_gray, scaleFactor=1.2, minNeighbors=5, minSize=(30, 30))

            if len(faces) == 1:
                x, y, w, h = faces[0]
                cropped_face = img[y:y+h, x:x+w]  # Crop the face from the image
                faces_found += 1
                cv2.imwrite(os.path.join(dir_name, f'{faces_found}.jpg'), cropped_face)
        except Exception as e:
            print(f"Error processing image {url}: {e}")

# URL of the Haarcascade Frontal Face Classifier
trained_haarcascade_url = "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml"

# Load Haarcascade Classifier into a temporary file
response = requests.get(trained_haarcascade_url)
with NamedTemporaryFile(delete=False, suffix='.xml') as temp_file:
    temp_file.write(response.content)
    temp_file_path = temp_file.name

# Load the classifier from the temporary file
faceCascade = cv2.CascadeClassifier(temp_file_path)

# Process images for each celebrity
print("Processing Kanye West images")
process_and_save_images(kanye_images, 'dataset/train/kanye')

print("\nProcessing Ice Spice images")
process_and_save_images(ariana_images,'dataset/train/ariana')



print("\nProcessing Ice Spice images")
process_and_save_images(kanye_lookalike_images, 'dataset/validation/kanyelookalike')


print("\nProcessing Ice Spice images")
process_and_save_images(ariana_lookalike_images, 'dataset/validation/arianalookalike')


from PIL import Image, ImageEnhance, ImageFilter
import random

# adding blurs and brightness

def modify_image(img):
    # Random rotation
    img = img.rotate(random.randint(-30, 30))

    # Adjust brightness
    enhancer = ImageEnhance.Brightness(img)
    img = enhancer.enhance(random.uniform(0.5, 1.5))

    # Add blur
    img = img.filter(ImageFilter.GaussianBlur(radius=random.randint(0, 3)))

    return img


def modify_all_images_in_directory(source_directory, destination_directory):
    if not os.path.exists(destination_directory):
        os.makedirs(destination_directory)

    for filename in os.listdir(source_directory):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            img_path = os.path.join(source_directory, filename)
            img = Image.open(img_path)
            modified_img = modify_image(img)
            modified_img.save(os.path.join(destination_directory, filename))



source_dir = 'dataset/validation/kanyelookalike'
destination_dir = 'dataset/validation_modified/kanyelookalike'

source_dir2 = 'dataset/validation/arianalookalike'
destination_dir2 = 'dataset/validation_modified/arianalookalike'

modify_all_images_in_directory(source_dir, destination_dir)
modify_all_images_in_directory(source_dir2, destination_dir2)



Processing Kanye West images

Processing Ice Spice images

Processing Ice Spice images

Processing Ice Spice images


In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


def print_directory_contents(path):
    for dirpath, dirnames, filenames in os.walk(path):
        print(f"Directory: {dirpath}")
        for filename in filenames:
            print(f"  File: {filename}")

# Example usage
print_directory_contents('dataset/train')

#load the model

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freezing the layers
for layer in base_model.layers:
    layer.trainable = False

# Adding custom layers
x = Flatten()(base_model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=predictions)

#compile and customise
model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

#prepare dataset

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_generator = train_datagen.flow_from_directory(
    'dataset/train',  # Path to the main directory
    target_size=(224, 224),
    batch_size=10,
    class_mode='binary'
)


validation_generator = test_datagen.flow_from_directory(
    'dataset/train',
    target_size=(224, 224),
    batch_size=10,
    class_mode='binary')


#train the model

history = model.fit(
    train_generator,
    steps_per_epoch=1,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//validation_generator.batch_size
)



#evaluate the model

test_generator = test_datagen.flow_from_directory(
    'dataset/validation',
    target_size=(224, 224),
    batch_size=10,
    class_mode='binary')

test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy:', test_accuracy)



#evaluate the model

test_generator = test_datagen.flow_from_directory(
    'dataset/validation_modified',
    target_size=(224, 224),
    batch_size=10,
    class_mode='binary')

test_loss, test_accuracy = model.evaluate(test_generator)
print('Test accuracy with blurs and brightness adjusted:', test_accuracy)








Directory: dataset/train
Directory: dataset/train/ariana
  File: 6.jpg
  File: 7.jpg
  File: 5.jpg
  File: 3.jpg
  File: 2.jpg
  File: 4.jpg
  File: 10.jpg
  File: 1.jpg
  File: 9.jpg
  File: 8.jpg
Directory: dataset/train/kanye
  File: 6.jpg
  File: 7.jpg
  File: 5.jpg
  File: 3.jpg
  File: 2.jpg
  File: 4.jpg
  File: 10.jpg
  File: 1.jpg
  File: 9.jpg
  File: 8.jpg
58889256/58889256 [==============================] - 0s 0us/step
Found 20 images belonging to 2 classes.
Found 20 images belonging to 2 classes.
Epoch 1/10
1/1 [==============================] - 28s 28s/step - loss: 0.9855 - accuracy: 0.4000 - val_loss: 0.9828 - val_accuracy: 0.5000
Epoch 2/10
1/1 [==============================] - 26s 26s/step - loss: 0.8336 - accuracy: 0.6000 - val_loss: 0.5101 - val_accuracy: 0.6500
Epoch 3/10
1/1 [==============================] - 26s 26s/step - loss: 0.4972 - accuracy: 0.7000 - val_loss: 0.2262 - val_accuracy: 1.0000
Epoch 4/10
1/1 [==============================] - 26s 26s/step - los